# R-VonFrey up/down calculator

### R script is developend in Andrei V. Chernov's Lab at the University of California San Diego.

#### Version 0.1.beta https://github.com/chernov-lab/VonFreyTest

#### GNU GENERAL PUBLIC LICENSE Applies

#### The observation format complies with Drs. T.L. Yaksch's (UCSD) lab and V.I. Shubayev's (UCSD) lab records

##### Parametric data are from Chaplan et al, 1994: 
S R Chaplan  1 , F W Bach, J W Pogrel, J M Chung, T L Yaksh
Quantitative assessment of tactile allodynia in the rat paw
J Neurosci Methods, 1994 Jul;53(1):55-63. doi: 10.1016/0165-0270(94)90144-9.


## Provide file name of the excel file that includes the following tabs:

**data** (XO observations) and

**meta** (comparison setting)

### The **data** spreadsheet stores observation and last filament results
### The **meta** spreadsheet establishes comparison pairs between groups

#### Use the included test_data.xlsx file as a template

**data** table should have the following comumns:

'SEX', 'GROUP', 'PAW', 'TIMEPOINT', 'OBSERVATION', 'LAST', 'ANIMAL_ID'

**meta** table should contain columns 'TREATMENT' and 'REFERENCE'

**Currently only rat calculator is available!**

In [ ]:
data_file <- "test_data.xlsx"  # <<<- provide excel file name with VF data. File should be uploaded in the main folder

In [ ]:
if (file.exists(data_file)) {
    sprintf("Observation data: %s", data_file)
} else { 
    stop(sprintf("File %s does not exist", data_file))
}

## Set up test parameters:

In [ ]:
paw <- 'LEFT' #'RIGHT',  # define paw to use
sex <- "female"  # define sex of animals if needed. Assign "" is sex is not defined
species <- "rat" # define species of animal rat or mouse
post_hoc_method <- "bonferroni" # Allowed values include "holm", "hochberg", "hommel", "bonferroni", "BH", "BY", "fdr", "none". If you don't want to adjust the p value (not recommended), use p.adjust.method = "none".
label_id <- T # set TRUE if want to label animal IDs below threshold

### Define the results file

In [ ]:
res_file <- paste('result', data_file, sep='.')  # results file

#### Loading R libraries

In [ ]:
library(readr)
library(ggplot2)
library(ggpubr)
library(dplyr)
library(tidyverse)
library(rstatix)
library(DescTools)
library(readxl)
library(writexl)
library(lubridate)
library(emmeans)

### Define some custom functions

In [ ]:
`%notin%` <- Negate(`%in%`)

acPage = function(w1=12, h1=6, r1=1, c1=1, m1=2, m2=2, m3=2, m4=2){
  options(repr.plot.width=w1, repr.plot.height= h1) 
  par(mfrow=c(r1,c1), mar=c(m1,m2,m3,m4))
}

### Folder definitions

In [ ]:
main_folder = ""
data_folder <- "files"
par_folder <- "parameters"

sex <- toupper(sex)

if (species == "rat"){
    pain_threshold <- 5
    max_res <- 15
}

### Functions for Von Frey Test calculations

In [ ]:
XO_file <- paste(species, "final.previous.filaments.XO.csv", sep='.')
stat_file <- paste(species, "observation-statistics.csv", sep='.')

observation_table <- read.csv(file.path(par_folder, stat_file), row.names=1, stringsAsFactors=FALSE)
XO.table <- read.csv(file.path(par_folder, XO_file), row.names=1, stringsAsFactors=FALSE)

VonFrey = function(obs='', last=0){
    obs <- toupper(obs)
    res <- 0
    if (obs == '' | last == 0) { return (-1)} 
    else if (obs == 'OOOOO' & last == 5.18) { res <- max_res } 
    else if (obs == 'XXXX' & last == 3.61) { res <- 0.2 }
    else if (length(which(rownames(XO.table) == last)) == 0 | length(which(rownames(observation_table) == obs))== 0) { return (-1)}
    else {
        a <- substr(obs, nchar(obs)-1, nchar(obs)-1)
        prev <- XO.table[which(rownames(XO.table) == last), a]
        dif <- abs(last - prev)
        p50 <- last + dif * observation_table[which(rownames(observation_table) == obs),'STATISTIC']
        res <- (10**p50)/10000
        if (res > 15) { res <- max_res }
        if (res < 0.2) { res <- 0.20 }
        return(as.numeric(sprintf("%.2f", res)))
    }
}

TruncateObservation = function(obs = '', met = 2){
    obs <- toupper(obs)
    if (obs == '') { return ("") } 
    if (met == 2) { return (obs) }
    if (met == 1) {  # Jenny's short method
        len <- nchar(obs)  
        Xpos <- StrPos(obs, 'X') 
        if (is.na(Xpos)) { return (obs) }   
        obs1 <- substr(obs, Xpos, len) 
        return (obs1)
    }
}

## Read DATA and META files from main folder

In [ ]:
dt <- readxl::read_excel(data_file, sheet = "data") 
meta <- readxl::read_excel(data_file, sheet = "meta")

In [ ]:
dt <- dt %>% rename_with(toupper)
meta <- meta %>% rename_with(toupper)

# convert key parameters to uppercase
dt <- dt %>% dplyr::mutate (
                DATE = ymd(DATE),
                SEX = toupper(SEX), 
                OBSERVATION = toupper(OBSERVATION),
                PAW = toupper(PAW))
dt <- dt %>% dplyr::arrange(SEX, TIMEPOINT, PAW)
head(dt)
head(meta)

### filter by specific sex if **sex** is assigned

In [ ]:
if (sex != ""){ dt <- dt %>% filter (SEX == sex) }

### Calculate **Von Frey Test** values

In [ ]:
dt <- dt %>% dplyr::mutate(RESULT = mapply(VonFrey, OBSERVATION, LAST))

In [ ]:
head(dt, 5)

### Check for errors reported by Von Frey calculator
#### Negative RESULTs (-1) indicate an error most likely related to incorrect XO data entry

In [ ]:
dt %>% filter(RESULT < 0)

### Prepare dataset for ANOVA analysis and plotting

In [ ]:
dt <- dt %>% filter(RESULT > 0)
lp_dt_names <- c('SEX', 'GROUP', 'PAW', 'TIMEPOINT', 'RESULT', 'ANIMAL_ID')
lp_meta_names <- c('COMPARISON', 'TREATMENT', 'REFERENCE')

In [ ]:
lp <- dt %>% 
    select(all_of(lp_dt_names)) %>% 
        mutate(DAY = TIMEPOINT, 
        TIMEPOINT = as.integer(TIMEPOINT),
        DAY = factor(TIMEPOINT),
        GROUP = factor(GROUP),
    ID = row_number()) %>% 
    print

### Set up comparison pairs

In [ ]:
meta <- meta %>% 
    mutate(COMPARISON = mapply(sprintf, "%s-%s", TREATMENT, REFERENCE)) %>% 
    print

In [ ]:
mt <- meta %>% 
    select (TREATMENT, REFERENCE) %>% 
    t
comparisons <- lapply(seq_len(ncol(mt)), function(i) mt[,i])

### Filter by LEFT or RIGHT paw

In [ ]:
if (paw != "") { lp <- lp %>% filter(PAW == paw) }

In [ ]:
min_res <- lp %>% 
    select (RESULT) %>% 
    min
max_res <- lp %>% 
    select (RESULT) %>% 
    max

## Summary statistics
Compute some summary statistics (count, mean and sd) of the variable weight organized by groups:

# Two-way ANOVA

## Summary statistics
Compute the mean and the SD (standard deviation) of the score by groups:

In [ ]:
lp_stat <- lp %>%
    group_by(GROUP, DAY) %>%
    get_summary_stats(RESULT, type = "mean_se") %>% 
    mutate (TIMEPOINT = as.numeric(strtoi(DAY))) %>% 
    print

## Visualization
Create a box plot of the score by gender levels, colored by education levels:

In [ ]:
acPage(w1=15, h1=6)
bxp <- lp %>% ggboxplot(
    x = "GROUP", y = "RESULT",
    color = "GROUP", 
    palette = "jco",
    ) +
    theme_bw() +
    rremove("x.text") +
    geom_hline(yintercept=5, linetype="dashed", color = "orange", size=1) +
    facet_grid( ~ TIMEPOINT, labeller = "label_both")

plot(bxp)

In [ ]:
lp_outs <- lp %>%
    group_by(GROUP, DAY) %>%
    identify_outliers(RESULT) %>% 
    filter(is.extreme == TRUE) %>% 
    print()

lp <- lp %>% 
    anti_join(lp_outs, by = "ID") %>% 
    print()

### Build the linear model

In [ ]:
model  <- lm(RESULT ~ GROUP * DAY, data = lp)

## Normality assumption
The normality assumption can be checked by using one of the following two approaches:

* Analyzing the ANOVA model residuals to check the normality for all groups together. This approach is easier and it’s very handy when you have many groups or if there are few data points per group.

* Check normality for each group separately. This approach might be used when you have only a few groups and many data points per group.
In this section, we’ll show you how to proceed for both option 1 and 2.

### Create a QQ plot of residuals

In the QQ plot, as all the points fall approximately along the reference line, we can assume normality. This conclusion is supported by the Shapiro-Wilk test. If the p-value is not significant, we can assume normality.

In [ ]:
model %>% residuals %>% ggqqplot

### Compute Shapiro-Wilk test of normality

In [ ]:
model %>% residuals %>% shapiro_test

Check normality assumption by analyzing the model residuals. QQ plot and Shapiro-Wilk test of normality are used. QQ plot draws the correlation between a given data and the normal distribution.

In [ ]:
Shapiro <- lp %>% 
    group_by(DAY, GROUP) %>% 
    shapiro_test(RESULT)

Shapiro %>% filter( p < 0.05 ) %>% print()

Shapiro %>% filter( p > 0.05 ) %>% print()

Check if the scores are normally distributed (p > 0.05) for each group, as assessed by Shapiro-Wilk’s test of normality.

In [ ]:
ggqqplot(lp, "RESULT", ggtheme = theme_bw()) +
    facet_grid( GROUP ~ TIMEPOINT, labeller = "label_both")

### Calculate ANOVA statistics table with post hoc corrections 

In [ ]:
pwc <- lp %>% 
    group_by(DAY) %>%
    emmeans_test(RESULT ~ GROUP, model = model, 
            comparisons = comparisons,
            p.adjust.method = post_hoc_method) 

pwc_stat <- pwc

### List most significant differences between groups

In [ ]:
pwc %>% 
    filter(p.adj < 0.1) %>% 
    arrange (p.adj) %>% 
    print

### Add formatted P-value for graphics

In [ ]:
pwc$p.format <- p_format(pwc$p, accuracy = 0.0001, leading.zero = FALSE)
head(pwc)

In [ ]:
days <- lp %>% select(TIMEPOINT) %>% arrange %>% unique %>% as.list
days

## Plot all data on one graph

In [ ]:
acPage(w1=15, h1=10)

x_max <- max(lp$TIMEPOINT) + 1

p <- ggline(lp, 
            x = "TIMEPOINT",
            y = "RESULT", 
            size = 1.5,            
            color = "GROUP",
            shape = "GROUP",
            point.size = 5,
            linetype = "GROUP",
            ylim = c(0, max_res),
            xlim = c(0, x_max + 1),
            add = c("mean_se", "jitter" ),
            add.params = list(width = 0.1, shape = 15),
            palette = "GROUP"
            )  +
    theme_bw() +
    geom_point(size = 3, aes(color = GROUP, shape = GROUP)) +
    xlab("Time after IS injection, days") +
    ylab("Tactile threshold, g") +
    ggtitle( sprintf( "von Frey behavior tests in %s paws", tolower (paw))) +
    theme(text = element_text(size=20, color = "black", angle = 0, hjust = .5, vjust = 0, face = "italic"),
    axis.text.x = element_text(hjust=1)) +
    geom_hline(yintercept=5, linetype="dashed", color = "orange", size=1.2) +
    scale_y_continuous(breaks= 0 : round(max_res) * max_res/3, expand = expansion(mult = c(0, 0.1))  ) +
    scale_x_continuous(breaks= 0 : x_max * 1) 
plot(p)

### Plot this graph into a PDF file

In [ ]:
pdf(sprintf("vonFrey plot.pdf"), width = 15, height = 10)
plot(p)
dev.off()

### Ensure X axis coordinates correctly mapped

In [ ]:
pwc <- pwc %>% 
    mutate(x = as.numeric(DAY), 
           xmin = as.numeric(strtoi(DAY)), 
           xmax = as.numeric(strtoi(DAY)) + 0.32) %>% 
    arrange(x)

## Plot graphs for individual comparisons and save in PDF files

In [ ]:
acPage(w1=12, h1=7)

print("Significance scores: * < 0.05; ** < 0.005; *** < 0.0005; **** < 0.00005")

for (i in 1:nrow(meta)){

lp_plot <- lp %>% filter ( GROUP == meta$TREATMENT[i] | GROUP == meta$REFERENCE[i] ) 
pwc.f <- pwc %>% filter( group1 == meta$TREATMENT[i] & group2 == meta$REFERENCE[i])
    
pp <- ggline(
            lp_plot, 
            y = "RESULT", 
            x = "TIMEPOINT", 
            color = "GROUP", 
            ylim = c(0, max_res + 1),
            linetype = "GROUP",
            size = 1.5, binwidth=0.8, 
            add = c("mean_se"),
            palette = c("red3", "blue1"),
            ) +  scale_x_continuous(breaks = 0:21*1) +
    stat_pvalue_manual( 
            pwc.f, 
            label = "p.adj.signif",
            position = position_dodge(0.6), 
            remove.bracket = T,
            size = 11,
            y.position = 15.5,
            hide.ns = T,
            color = "black"
            ) +
    geom_point(aes(shape = GROUP, color = GROUP, size = 10), alpha = 5/10, show.legend = F) +
    xlab(label = "Time, days") +
    ylab(label = "Tactile threshold, g") +
    ggtitle( sprintf( "%s vs %s by von Frey in %s %s paws. Mean\u00B1SE, ANOVA, %s post-hoc", 
                     meta$TREATMENT[i], meta$REFERENCE[i], species, tolower (paw), str_to_title(post_hoc_method))) +
    theme_bw() +
    geom_hline(yintercept=pain_threshold, linetype="dashed", color = "orange", size=1.2) +
    theme(text = element_text(size=16, color = "black", angle = 0, hjust = .5, vjust = 0, face = "italic")) +
    geom_text(aes(label=ifelse(label_id & RESULT <= pain_threshold,as.character(ANIMAL_ID),'')),hjust = 0, vjust = 0, size = 4, check_overlap = T)

    plot(pp)

    pdf(sprintf("vonFrey plot %s vs %s in %s %s paw.pdf", meta$TREATMENT[i], meta$REFERENCE[i], species, tolower (paw)), width = 12, height = 7)
        plot(pp)
    dev.off()
}

In [ ]:
pwc <- pwc %>% 
    mutate (DAY = strtoi(DAY)) %>% 
    arrange (group1, group2, DAY)

for (i in 1:nrow(meta)){
    sprintf("Significance scores for %s vs %s, post-hoc %s", meta$TREATMENT[i], meta$REFERENCE[i], post_hoc_method) %>% print
    pwc %>% 
    filter(group1 == meta$TREATMENT[i] & group2 == meta$REFERENCE[i]) %>% 
    arrange (DAY) %>% 
    select (DAY, group1, group2, p, p.adj, p.adj.signif) %>%
    print
}

### Kruskal-Wallis test 

Kruskal-Wallis test is a non-parametric alternative to the one-way ANOVA test. It extends the two-samples Wilcoxon test in the situation where there are more than two groups to compare. 

In [ ]:
res.kruskal <- lp %>% 
    group_by(TIMEPOINT) %>% 
    kruskal_test(RESULT ~ GROUP)
res.kruskal

## Effect size
The eta squared, based on the H-statistic, can be used as the measure of the Kruskal-Wallis test effect size. It is calculated as follow : 

    eta2[H] = (H - k + 1)/(n - k); 

where H is the value obtained in the Kruskal-Wallis test; k is the number of groups; n is the total number of observations (M. T. Tomczak and Tomczak 2014).

The eta-squared estimate assumes values from 0 to 1 and multiplied by 100 indicates the percentage of variance in the dependent variable explained by the independent variable.

The interpretation values commonly in published literature are: 0.01- < 0.06 (small effect), 0.06 - < 0.14 (moderate effect) and >= 0.14 (large effect).

In [ ]:
lp %>% 
    group_by(TIMEPOINT) %>% 
    kruskal_effsize(RESULT ~ GROUP)

## Multiple pairwise-comparisons
From the output of the Kruskal-Wallis test, we know that there is a significant difference between groups, but we don’t know which pairs of groups are different.

A significant Kruskal-Wallis test is generally followed up by Dunn’s test to identify which groups are different.

### Pairwise comparisons using Dunn’s test:

In [ ]:
# Pairwise comparisons
pwc_dunn <- lp %>% 
    group_by(TIMEPOINT) %>% 
    dunn_test(RESULT ~ GROUP, p.adjust.method = post_hoc_method) 
pwc_dunn %>% filter(p.adj.signif != 'ns') %>% head(3)

It’s also possible to use the Wilcoxon’s test to calculate pairwise comparisons between group levels with corrections for multiple testing.

In [ ]:
pwc_wilcox <- lp %>% 
    group_by(TIMEPOINT) %>% 
    wilcox_test(RESULT ~ GROUP, p.adjust.method = post_hoc_method)

(pwc_wilcox %>% filter(p.adj.signif != 'ns'))

## Report
There was a statistically significant differences between treatment groups as assessed using the Kruskal-Wallis test (p = 0.018). Pairwise Wilcoxon test between groups showed that only the difference between trt1 and trt2 group was significant (Wilcoxon’s test, p = 0.027)

In [ ]:
# Visualization: box plots with p-values

acPage(w1=15, h1=5)
#res.kruskal <- lp %>% group_by(TIMEPOINT) %>% kruskal_test(RESULT ~ GROUP)
#pwc_dunn <- lp %>%  group_by(TIMEPOINT) %>% dunn_test(RESULT ~ GROUP, p.adjust.method = "bonferroni") 
pwc_dunn <- pwc_dunn %>% add_xy_position(x = "GROUP")

p1 <- lp %>% 
        ggboxplot(x = "GROUP", y = "RESULT",  add = c("mean_se"), color = "GROUP") +
        stat_pvalue_manual(pwc_dunn, hide.ns = TRUE) +
        labs(
        subtitle = get_test_label(res.kruskal, detailed = T, correction = post_hoc_method),
        caption = get_pwc_label(pwc_dunn),
        type = "expresion"
        ) +
geom_point(aes(shape = GROUP, color = GROUP, size = 10), alpha = 5/10, show.legend = F) +
xlab(label = "Time, days") +
ylab(label = "Tactile threshold, g") +
theme_bw() +
geom_hline(yintercept=pain_threshold, linetype="dashed", color = "orange", size=1.2) +
theme(text = element_text(size=16, color = "black", angle = 0, hjust = .5, vjust = 0, face = "italic")) +
facet_grid( ~ TIMEPOINT, labeller = "label_both") +
geom_text(aes(label=ifelse(label_id & RESULT<=pain_threshold,as.character(ANIMAL_ID),'')), hjust = 1, vjust = 1.5, size = 4, check_overlap = T)

plot(p1)

### Save results

In [ ]:
sprintf("Save results in %s file", res_file)

In [ ]:
write_xlsx(list(data = dt, meta = meta, ANOVA = pwc, Kruskal = res.kruskal, Dunn = pwc_dunn, Wilcox = pwc_wilcox), res_file)

In [ ]:
sessionInfo()